## 1. Libraries and settings

In [12]:
import numpy as np
import pandas as pd 
from pylab import mpl, plt


import optuna

import torch
import torch.nn as nn
import torch.optim as optim


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight


import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-darkgrid')
mpl.rcParams['font.family'] = 'serif'
%matplotlib inline

import warnings
warnings.simplefilter("ignore", UserWarning)

import vectorbtpro as vbt




In [13]:
df = pd.read_csv('2ySOLdata1h.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
df.set_index('timestamp', inplace=True)

pd.set_option('future.no_silent_downcasting', True)
df['signal'] = df['signal'].replace({'SignalNone': 1, 'SignalLong': 2, 'SignalShort': 0})
df = df.ffill()

In [14]:
data = vbt.Data.from_data(df)
features = data.run("talib", mavp=vbt.run_arg_dict(periods=14))
data.data['symbol'] = pd.concat([data.data['symbol'], features], axis=1)
data.data['symbol'].drop(['Open', 'High', 'Low'], axis=1, inplace=True)
# This will drop columns from the DataFrame where all values are NaN
data.data['symbol'] = data.data['symbol'].dropna(axis=1, how='all')

open_price = data.get('Open')
high_price = data.get('High')
low_price = data.get('Low')
close_price = data.get('Close')

data.data['symbol'] = data.data['symbol'].dropna()
predictor_list = data.data['symbol'].drop('signal', axis=1).columns.tolist()


X = data.data['symbol'][predictor_list]

y = data.data['symbol']['signal']

X.columns = X.columns.astype(str)



In [15]:
# Split the data into a training set and a test set
# Assuming X is a DataFrame or a NumPy array
indices = np.arange(X.shape[0])

# First, split your data into a training+validation set and a separate test set
X_train_val, X_test, y_train_val, y_test, indices_train_val, indices_test = train_test_split(X, y, indices, test_size=0.3, shuffle=False)

# Then, split the training+validation set into a training set and a validation set
X_train, X_val, y_train, y_val, indices_train, indices_val = train_test_split(X_train_val, y_train_val, indices_train_val, test_size=0.2, shuffle=False)  # 0.2 here means 20% of the original data, or 25% of the training+validation set

# Now, `indices_val` holds the indices of your original dataset that were used for the validation set.


scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_val_scaled = scaler.fit_transform(X_val)

In [16]:
timestep = 20

def create_sequences(input_data, timestep):
    sequences = []
    data_len = len(input_data)
    for i in range(data_len - timestep):
        seq = input_data[i:(i + timestep)]
        sequences.append(seq)
    return np.array(sequences)

X_train_list = create_sequences(X_train_scaled, timestep)
X_val_list = create_sequences(X_val_scaled, timestep)
X_test_list = create_sequences(X_test_scaled, timestep)
y_train_seq_ar = y_train[timestep:]
y_val_seq_ar = y_val[timestep:]
y_test_seq_ar = y_test[timestep:]

In [17]:
# Convert to numpy arrays
x_train_ar = np.array(X_train_list)
y_train_seq = np.array(y_train_seq_ar).astype(int)
x_val_ar = np.array(X_val_list)  
y_val_seq = np.array(y_val_seq_ar).astype(int)
x_test_ar = np.array(X_test_list)  
y_test_seq = np.array(y_test_seq_ar).astype(int)

In [18]:
# Convert to tensors
X_train_tensor = torch.tensor(x_train_ar, dtype=torch.float32) # .to(device)
y_train_tensor = torch.tensor(y_train_seq, dtype=torch.long)
X_val_tensor = torch.tensor(x_val_ar, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val_seq, dtype=torch.long)
X_test_tensor = torch.tensor(x_test_ar, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_seq, dtype=torch.long)


In [19]:
# Check for MPS (GPU on M1 Mac) availability and set it as the device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
# print(f"Using device: {device}")

In [20]:
# Compute class weights
# Convert y_train to a numpy array if it's a tensor
if isinstance(y_train_seq, torch.Tensor):
    y_train_seq_np = y_train_seq.cpu().numpy()
else:
    y_train_seq_np = y_train_seq  # Assuming y_train_seq is already a numpy array or similar

class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_seq_np), y=y_train_seq_np)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)


# Move class weights to the same device as your model and data
class_weights_tensor = class_weights_tensor.to(device)  # device could be 'cpu' or 'cuda'

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.hidden_dim = hidden_dim
        self.linear = nn.Linear(hidden_dim, 1)

    def forward(self, lstm_output):
        # lstm_output shape: [batch_size, seq_length, hidden_dim]
        weights = torch.tanh(self.linear(lstm_output))
        weights = F.softmax(weights, dim=1)
        
        # Context vector with weighted sum
        context = weights * lstm_output
        context = torch.sum(context, dim=1)
        return context, weights

class BiLSTMClassifierWithAttention(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim, dropout_rate):
        super(BiLSTMClassifierWithAttention, self).__init__()
        
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        
        # Convolutional Layer
        self.conv1 = nn.Conv1d(in_channels=input_dim, out_channels=hidden_dim, kernel_size=3, stride=1, padding=1)
        
        # Batch Normalization Layer for Conv1d
        self.bn_conv1 = nn.BatchNorm1d(hidden_dim)
        
        # LSTM Layer
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True)
        
        # Attention Layer
        self.attention = Attention(hidden_dim * 2)  # For bidirectional LSTM
        
        # Dropout layer
        self.dropout = nn.Dropout(dropout_rate)
        
        # Fully connected layers
        self.fc1 = nn.Linear(hidden_dim * 2, hidden_dim)  # Adjusted for attention context vector
        
        # Batch Normalization Layer for FC1
        self.bn_fc1 = nn.BatchNorm1d(hidden_dim)
        
        self.fc2 = nn.Linear(hidden_dim, output_dim)  # Output layer
        
        # Additional Dropout for the fully connected layer
        self.dropout_fc = nn.Dropout(dropout_rate / 2)

    def forward(self, x):
        # Reshape x for Conv1d
        x = x.permute(0, 2, 1)
        
        # Convolutional layer
        x = self.conv1(x)
        x = self.bn_conv1(x)
        x = F.relu(x)
        
        # Reshape back for LSTM
        x = x.permute(0, 2, 1)
        
        # Initialize hidden and cell states
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_dim).to(x.device)
        
        # LSTM layer
        out, _ = self.lstm(x, (h0, c0))
        
        # Applying attention mechanism to LSTM outputs
        context, _ = self.attention(out)
        
        # Fully connected layers using the context vector from attention
        out = self.fc1(context)
        out = self.bn_fc1(out)
        out = F.relu(out)
        out = self.dropout_fc(out)
        out = self.fc2(out)
        
        return out


In [ ]:
vbt.settings.set_theme('dark')
vbt.settings['plotting']['layout']['width'] = 500
vbt.settings['plotting']['layout']['height'] = 250

num_epochs = 30
num_trials = 5 # X_train_tensor.shape[2]  # number of trials to run
# lets validate our technical indicators with the signal
from sklearn.metrics import accuracy_score

def objective(trial):
    # Suggest hyperparameters
    hidden_dim = 32 # trial.suggest_categorical('hidden_dim', [16, 32, 64])
    num_layers = 2 # trial.suggest_int('num_layers', 1, 3)
    lr = 1e-3 # trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    step_size = 25 # trial.suggest_int('step_size', 10, 100)
    gamma = 0.85 # trial.suggest_float('gamma', 0.85, 0.99)
    dropout_rate = 0.1 # trial.suggest_float('dropout_rate', 0.1, 0.4)

    ''' multi feature selection'''
    # Suggest a boolean flag for each feature to decide if it should be included
    num_features = X_train_tensor.shape[2]  # assuming the last dimension is the feature dimension
    included_features = [trial.suggest_categorical(f'include_feature_{i}', [True, False]) for i in range(num_features)]
    included_features_idx = [i for i, f in enumerate(included_features) if f]
    # If no features are selected, we can either skip this trial or select a default feature
    if not included_features_idx:
        return None  # Or handle this case as you see fit
    # Use a selection of features to create new tensors
    X_train_selected = X_train_tensor[:, :, included_features_idx]
    X_val_selected = X_val_tensor[:, :, included_features_idx]
    X_test_selected = X_test_tensor[:, :, included_features_idx]
    
    ''' single feature selection'''
    # Use only the selected feature to create new tensors
    # feature_idx = trial.suggest_int('feature_idx', 0, X_train_tensor.shape[2] - 1)
    # X_train_selected = X_train_tensor[:, :, feature_idx:feature_idx+1]
    # X_val_selected = X_val_tensor[:, :, feature_idx:feature_idx+1]
    # X_test_selected = X_test_tensor[:, :, feature_idx:feature_idx+1]

 
    
    # Move the selected feature tensors to the GPU
    y_train_tensor_gpu = y_train_tensor.to(device)
    X_train_selected_gpu = X_train_selected.to(device)
    y_train_tensor_gpu = y_train_tensor.to(device)
    X_val_selected_gpu = X_val_selected.to(device)

    # Initialize model and move it to the MPS device
    model = BiLSTMClassifierWithAttention(input_dim=X_train_selected.shape[-1], hidden_dim=hidden_dim, num_layers=num_layers, output_dim=len(np.unique(y_train_tensor.cpu().numpy())), dropout_rate=dropout_rate).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
    criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)


    # Training loop
    model.train()
    for epoch in range(num_epochs):  # use a small number of epochs for demonstration
        optimizer.zero_grad()
        output = model(X_train_selected_gpu)
        loss = criterion(output, y_train_tensor_gpu)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        # # Validation step
        # if epoch % 10 == 0:
        #     model.eval()
        #     with torch.no_grad():
        #         val_output = model(X_val_selected_gpu)
        #         val_loss = criterion(val_output, y_val_tensor_gpu)
        #         # Convert model outputs to predicted classes
        #         _, predicted_classes = torch.max(val_output, 1)
                
        #         # Convert tensors to numpy arrays for compatibility with sklearn
        #         predicted_classes = predicted_classes.cpu().numpy()
        #         true_classes = y_val_tensor_gpu.cpu().numpy()
                
        #         # Filter out 'hold' predictions and labels
        #         buy_sell_filter = (true_classes != 1) & (predicted_classes != 1)
        #         filtered_true_classes = true_classes[buy_sell_filter]
        #         filtered_predicted_classes = predicted_classes[buy_sell_filter]
                
        #         if len(filtered_predicted_classes) > 0 and len(filtered_true_classes) > 0:
        #             accuracy = accuracy_score(filtered_true_classes, filtered_predicted_classes)
        #             # print(f"Validation Accuracy (excluding 'hold'): {accuracy}")
        #         else:
        #             # print("Filtered classes are empty. Skipping accuracy calculation.")
        #             accuracy = 0  # or some default value
                
        #     model.train()

    # return val_loss.item()
        
    model.eval()
    with torch.no_grad():
        y_test_pred = model(X_val_selected_gpu)
        probabilities = torch.softmax(y_test_pred, dim=1)
        _, predicted_labels = torch.max(probabilities, 1)
        predicted_labels_numpy = predicted_labels.cpu().numpy()

    
    # Use predicted labels to simulate a trading strategy
    adjusted_indices_val = indices_val[timestep:]
    adjusted_indices_test = indices_test[timestep:] 
    df_split = data.data['symbol'].iloc[adjusted_indices_val].copy()


    df_split.loc[:, "signal"] = predicted_labels_numpy
    signal = df_split['signal']
    entries = signal == 2
    exits = signal == 0
    pf = vbt.Portfolio.from_signals(
        close=df_split.Close, 
        long_entries=entries, 
        long_exits=exits,
        size=100,
        size_type='value',
        init_cash='auto'
    )
    stats = pf.stats()
    total_return = stats['Total Return [%]']
    orders = stats['Total Orders']
    # max_drawdown = stats['Max Drawdown [%]']
    # sortino_ratio = stats['Sortino Ratio']
    calmer_ratio = stats['Calmar Ratio']

    if orders < 8:
        print(f"Only {orders} trades were made")
        calmer_ratio = 0.0
    else:
        pf.plot({"orders", "cum_returns"}).show()
        print(f"Total returns: {total_return} %")

    # Return the loss as the objective to minimize it
    # return accuracy
    
    # # Return the total return as the objective to maximize it
    return calmer_ratio + total_return



# Before running the study, ensure your data tensors are on the CPU as Optuna will handle moving them to the GPU
X_train_tensor = X_train_tensor.cpu()
y_train_tensor = y_train_tensor.cpu()
X_val_tensor = X_val_tensor.cpu()
y_val_tensor = y_val_tensor.cpu()
X_test_tensor = X_test_tensor.cpu()
y_test_tensor = y_test_tensor.cpu()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=num_trials)

print('Best trial:', study.best_trial.params)


In [ ]:
# Assuming 'study' is your completed Optuna study
# Get all completed trials
completed_trials = [trial for trial in study.trials if trial.state == optuna.trial.TrialState.COMPLETE]


# Get all completed trials
# completed_trials = study.trials
# completed_trials

# # Sort the trials based on their performance (assuming higher return is better)
# # Note: Adjust the sorting key based on your actual return metric if necessary
sorted_trials = sorted(completed_trials, key=lambda trial: trial.value, reverse=True)

# Get the top N performing feature indices
top_n = 5  # For example, top 5 features
top_n_features = [trial.params['feature_idx'] for trial in sorted_trials[:top_n]]

# # print("Top performing feature indices:", top_n_features)

# Map the indices to names
top_performing_feature_names = [predictor_list[idx] for idx in top_n_features]
top_performing_feature_names
# top_n_features


In [ ]:
# Assuming 'study' is your completed Optuna study


best_trial = study.best_trial

print(f"Best trial number: {best_trial.number}")
print("Best trial's parameters:", best_trial.params)
print("Best trial's objective value:", best_trial.value)


# Assuming best_trial.params is your dictionary
params = best_trial.params

# Extracting feature indices for which the value is True
included_feature_indices = [int(key.split('_')[-1]) for key, value in params.items() if value]

print("Included feature indices:", included_feature_indices)

# Map the indices to names
top_performing_feature_names = [predictor_list[idx] for idx in included_feature_indices]

print("Top performing feature names:", top_performing_feature_names)

In [ ]:


num_trials_2 = 5
num_epochs_2 = 10


def objective_2(trial):
    # Suggest hyperparameters
    hidden_dim = trial.suggest_categorical('hidden_dim', [16, 32, 64])
    num_layers = trial.suggest_int('num_layers', 1, 3)
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    step_size = trial.suggest_int('step_size', 10, 100)
    gamma = trial.suggest_float('gamma', 0.85, 0.99)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.4)
    # feature_idx = trial.suggest_int('feature_idx', 0, X_train_tensor.shape[2] - 1)
    # Suggest a boolean flag for each feature to decide if it should be included
    # num_features = X_train_tensor.shape[2]  # assuming the last dimension is the feature dimension
    # included_features = [trial.suggest_categorical(f'include_top_feature_{i}', [True, False]) for i in top_n_features]

    # included_features_idx = [i for i, f in enumerate(included_features) if f]
    
    # If no features are selected, we can either skip this trial or select a default feature
    # if not included_features_idx:
    #     return None  # Or handle this case as you see fit
    
    
    # Use only the selected features to create new tensors
    X_train_selected = X_train_tensor[:, :, included_feature_indices] # top_n_features]
    X_val_selected = X_val_tensor[:, :, included_feature_indices] # top_n_features]
    X_test_selected = X_test_tensor[:, :, included_feature_indices] # top_n_features]

    # Initialize model and move it to the MPS device
    model_2 = BiLSTMClassifierWithAttention(input_dim=X_train_selected.shape[-1], hidden_dim=hidden_dim, num_layers=num_layers, output_dim=len(np.unique(y_train_tensor.cpu().numpy())), dropout_rate=dropout_rate).to(device)
    optimizer = optim.Adam(model_2.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
    criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)

    X_train_tensor_gpu = X_train_tensor.to(device)
    y_train_tensor_gpu = y_train_tensor.to(device)
    X_val_tensor_gpu = X_val_tensor.to(device)
    y_val_tensor_gpu = y_val_tensor.to(device)
    X_test_tensor_gpu = X_test_tensor.to(device)
    y_test_tensor_gpu = y_test_tensor.to(device)
    
    # Move the selected feature tensors to the GPU
    X_train_selected_gpu = X_train_selected.to(device)
    y_train_tensor_gpu = y_train_tensor.to(device)
    X_test_selected_gpu = X_test_selected.to(device)
    X_val_selected_gpu = X_val_selected.to(device)

    # Training loop
    model_2.train()
    for epoch in range(num_epochs_2):  # use a small number of epochs for demonstration
        optimizer.zero_grad()
        output = model_2(X_train_selected_gpu)
        loss = criterion(output, y_train_tensor_gpu)
        loss.backward()
        optimizer.step()
        scheduler.step()
        # # Validation step
        if epoch % 20 == 0:
            print(f"Epoch {epoch} loss: {loss.item()}")
        
        
    model_2.eval()
    with torch.no_grad():
        y_test_pred = model_2(X_test_selected_gpu) # (X_test_selected_gpu)
        probabilities = torch.softmax(y_test_pred, dim=1)
        _, predicted_labels = torch.max(probabilities, 1)
        predicted_labels_numpy = predicted_labels.cpu().numpy()

    # Use predicted labels to simulate a trading strategy
    adjusted_indices_val = indices_val[timestep:]
    adjusted_indices_test = indices_test[timestep:] 
    df_split = data.data['symbol'].iloc[adjusted_indices_test].copy()
    
    df_split.loc[:, "signal"] = predicted_labels_numpy
    signal = df_split['signal']
    entries = signal == 2
    exits = signal == 0
    pf = vbt.Portfolio.from_signals(
        close=df_split.Close, 
        long_entries=entries, 
        long_exits=exits,
        size=100,
        size_type='value',
        init_cash='auto'
    )

    stats = pf.stats()
    total_return = stats['Total Return [%]']
    orders = stats['Total Orders']
    calmer_ratio = stats['Calmar Ratio']

    if orders < 10:
        print(f"Only {orders} trades were made")
        calmer_ratio = 0.0
    else:
        pf.plot({"orders", "cum_returns"}, settings=dict(bm_returns=False)).show()
    
    # Return the negative total return as the objective to maximize it
    return calmer_ratio

# Before running the study, ensure your data tensors are on the CPU as Optuna will handle moving them to the GPU
X_train_tensor = X_train_tensor.cpu()
y_train_tensor = y_train_tensor.cpu()
X_val_tensor = X_val_tensor.cpu()
y_val_tensor = y_val_tensor.cpu()
X_test_tensor = X_test_tensor.cpu()
y_test_tensor = y_test_tensor.cpu()

study_2 = optuna.create_study(direction='maximize')
study_2.optimize(objective_2, n_trials=num_trials_2)

print(f'Best trial:', study_2.best_trial.number, study_2.best_trial.params)


In [ ]:
# Assuming 'study_2' is your completed Optuna study


best_trial_2 = study_2.best_trial

print(f"Best trial number: {best_trial_2.number}")
print("Best trial's parameters:", best_trial_2.params)
print("Best trial's objective value:", best_trial_2.value)




# Assuming best_trial.params is your dictionary
params_2 = best_trial_2.params
params_2


# Extracting feature indices for which the value is True
# included_feature_indices = [int(key.split('_')[-1]) for key, value in params_2.items() if value]

# print("Included feature indices:", included_feature_indices)

# # Map the indices to names
# top_performing_feature_names = [predictor_list[idx] for idx in included_feature_indices]

# print("Top performing feature names:", top_performing_feature_names)